In [1]:
import pandas as pd
import numpy as np
# import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

In [3]:
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
df.drop(labels=df[df['tenure'] == 0].index, axis=0, inplace=True)
df[df['tenure'] == 0].index

Index([], dtype='int64')

In [5]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [6]:
def object_to_int(dataframe_series):
    if dataframe_series.dtype=='object':
        dataframe_series = LabelEncoder().fit_transform(dataframe_series)
    return dataframe_series

In [7]:
df = df.apply(lambda x: object_to_int(x))
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5365,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,2,29.85,2504,0
1,3953,1,0,0,0,34,1,0,0,2,...,2,0,0,0,1,0,3,56.95,1465,0
2,2558,1,0,0,0,2,1,0,0,2,...,0,0,0,0,0,1,3,53.85,156,1
3,5524,1,0,0,0,45,0,1,0,2,...,2,2,0,0,1,0,0,42.30,1399,0
4,6500,0,0,0,0,2,1,0,1,0,...,0,0,0,0,0,1,2,70.70,924,1


In [9]:
X = df.drop(columns = ['Churn'])
y = df['Churn'].values

In [10]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer, f1_score

# Định nghĩa hàm đánh giá F1 score
f1_scorer = make_scorer(f1_score, average='macro')

# Định nghĩa không gian tìm kiếm các tham số
param_space = {
    'C': Real(1e-6, 1e+6, prior='log-uniform'),  # Tham số điều chuẩn
    'solver': Categorical(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),  # Thuật toán tối ưu
    'max_iter': Integer(100, 1000),  # Số lượng vòng lặp tối đa
}

# Thiết lập mô hình Logistic Regression
lr = LogisticRegression(random_state=42)

# Thiết lập BayesSearchCV
opt_lr = BayesSearchCV(
    estimator=lr,
    search_spaces=param_space,
    n_iter=50,  # Số lượng lần lặp của Bayesian optimization
    scoring=f1_scorer,
    cv=5,  # Sử dụng 5-fold cross-validation
    n_jobs=-1,  # Sử dụng tất cả các CPU cores có sẵn
    random_state=42
)

# Thực hiện tìm kiếm tham số tối ưu
opt_lr.fit(X, y)

# In ra tham số tốt nhất và F1 score tương ứng
print("Tham số tối ưu của Logistic Regression:", opt_lr.best_params_)
print("F1 score tốt nhất của Logistic Regression:", opt_lr.best_score_)


Tham số tối ưu của Logistic Regression: OrderedDict([('C', 11185.625288472094), ('max_iter', 895), ('solver', 'lbfgs')])
F1 score tốt nhất của Logistic Regression: 0.7318916872138805


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 40, stratify=y)

In [13]:
best_params = opt_lr.best_params_

In [14]:
best_params

OrderedDict([('C', 11185.625288472094),
             ('max_iter', 895),
             ('solver', 'lbfgs')])

In [15]:
# Import các thư viện cần thiết
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# Tạo mô hình Logistic Regression
clf_lr = LogisticRegression(**best_params)

# Huấn luyện mô hình
clf_lr.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred_lr = clf_lr.predict(X_test)

# Đánh giá mô hình bằng F1-score
f1_lr = f1_score(y_test, y_pred_lr, average='weighted')  # Dùng average='weighted' cho bài toán đa lớp
print(f'Logistic Regression F1 Score: {f1_lr:.2f}')

# Báo cáo chi tiết
print('Logistic Regression Classification Report:')
print(classification_report(y_test, y_pred_lr))

print('Logistic Regression Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_lr))

# Nếu bạn muốn lưu mô hình để sử dụng sau này, bạn có thể sử dụng thư viện joblib
import joblib
joblib.dump(clf_lr, 'logistic_regression_model.pkl')


Logistic Regression F1 Score: 0.81
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.90      0.87      1033
           1       0.67      0.57      0.61       374

    accuracy                           0.81      1407
   macro avg       0.76      0.73      0.74      1407
weighted avg       0.80      0.81      0.81      1407

Logistic Regression Confusion Matrix:
[[927 106]
 [161 213]]


['logistic_regression_model.pkl']